In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from scipy.stats import poisson,skellam

In [2]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

In [3]:
predictor = pickle.load(open(os.path.join('predictor/plk_objects/predictor.plk'),'rb'))

In [4]:
mci_whu = simulate_match(predictor, 'Man City', 'West Ham', max_goals=10)

In [5]:
mci_whu

array([[9.77709130e-03, 9.96445330e-03, 5.07770289e-03, 1.72500293e-03,
        4.39514948e-04, 8.95875070e-05, 1.52173842e-05, 2.21557151e-06,
        2.82253665e-07, 3.19625107e-08, 3.25750200e-09],
       [3.52811217e-02, 3.59572268e-02, 1.83231442e-02, 6.22475912e-03,
        1.58601162e-03, 3.23280988e-04, 5.49126901e-05, 7.99500032e-06,
        1.01852643e-06, 1.15338314e-07, 1.17548585e-08],
       [6.36568438e-02, 6.48767233e-02, 3.30599899e-02, 1.12311769e-02,
        2.86160102e-03, 5.83287786e-04, 9.90775908e-05, 1.44251787e-05,
        1.83770172e-06, 2.08102030e-07, 2.12089966e-08],
       [7.65696321e-02, 7.80369641e-02, 3.97662076e-02, 1.35094207e-02,
        3.44207667e-03, 7.01607691e-04, 1.19175476e-04, 1.73513257e-05,
        2.21047944e-06, 2.50315519e-07, 2.55112407e-08],
       [6.90763342e-02, 7.04000695e-02, 3.58745859e-02, 1.21873546e-02,
        3.10522634e-03, 6.32946588e-04, 1.07512663e-04, 1.56532811e-05,
        1.99415633e-06, 2.25819009e-07, 2.30146462e-

In [6]:
print('Man City win Prob')
print(np.sum(np.tril(mci_whu, -1)))
print('Draw Prob')
print(np.sum(np.diag(mci_whu)))
print('West Ham win Prob')
print(np.sum(np.triu(mci_whu, 1)))

Man City win Prob
0.8390220829507811
Draw Prob
0.09591613856837271
West Ham win Prob
0.06376675446631597


In [7]:
from datetime import date

fixture = pd.read_csv("static/data/epl-2019-GMTStandardTime.csv")

today = date.today()
fixture['Date'] = pd.to_datetime(fixture['Date'])
future = fixture.loc[fixture['Date'] >= today]
homeTeam = future['Home Team'].iloc[:10]
awayTeam = future['Away Team'].iloc[:10]
dates = future['Date'].iloc[:10]

upcoming_ten_matches = pd.DataFrame(data = {'Date' : dates, 'HomeTeam': homeTeam, 'AwayTeam': awayTeam})

/Users/sb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  import sys


In [8]:
upcoming_ten_matches

,Date,HomeTeam,AwayTeam
260,2020-02-22,Chelsea,Spurs
261,2020-02-22,Burnley,Bournemouth
262,2020-02-22,Crystal Palace,Newcastle
263,2020-02-22,Sheffield Utd,Brighton
264,2020-02-22,Southampton,Aston Villa
265,2020-02-22,Leicester,Man City
266,2020-02-23,Man Utd,Watford
267,2020-02-23,Wolves,Norwich
268,2020-02-23,Arsenal,Everton
269,2020-02-24,Liverpool,West Ham


In [19]:
matches = []
home_prop = []
draw_prop = []
away_prop = []

for index, row in upcoming_ten_matches.iterrows():
    home = row['HomeTeam']
    if home == 'Spurs':
        home = 'Tottenham'
    if home == 'Sheffield Utd':
        home = 'Sheffield United'
    away = row['AwayTeam']
    if away == 'Spurs':
        away = 'Tottenham'
    if away == 'Sheffield Utd':
        away = 'Sheffield United'

    match = home + '_' + away
    matches.append(match)

    
    result = simulate_match(predictor, home, away, max_goals=10)
    home_prop.append(np.sum(np.tril(mci_whu, -1)))
    draw_prop.append(np.sum(np.diag(mci_whu)))
    away_prop.append(np.sum(np.triu(mci_whu, 1)))
    
result_pd = pd.DataFrame({'matches': matches, 'home': home_prop, 'draw': draw_prop, 'away': away_prop})
    

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error converting data to categorical: observation with value 'Man Utd' does not match any of the expected levels (expected: ['Arsenal', 'Aston Villa', ..., 'West Ham', 'Wolves'])
    goals ~ home + team + opponent
                   ^^^^